In [2]:
from IPython.core.interactiveshell import InteractiveShell
import georasters as gr
import rasterio.mask
import rasterio
import geopandas as gpd
import os
import sys
import glob
import re
import itertools
import collections
import multiprocessing
import requests
import pprint
import pickle
from pathlib import Path
from joblib import delayed, Parallel

# pyscience imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
import rioxarray as rxr
from rasterio.merge import merge
import os

/home/ar8787/.conda/envs/geo_shp/lib/python3.11/site-packages/libpysal/cg/alpha_shapes.py:38: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/home/ar8787/.conda/envs/geo_shp/lib/python3.11/site-packages/libpysal/cg/alpha_shapes.py:164: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/home/ar8787/.conda/envs/geo_shp/lib/python3.11/site-packages/libpysal/cg

## All Village Lvl


In [28]:
# Main paths
main_path = "/scratch/gpfs/ar8787/groupdata2/india_forest_land/C_Programs/_vcf_10_callaway_santanna_analysis/median_filter/by_state"
root = Path(main_path)

In [22]:
total_files = 35 

In [31]:
n_sample = 1
for i in np.arange(0, total_files):

    text1 = f"""

#-------------------------------------------------------------------------------

rm( list = ls() )
library(did)
library(augsynth)
library(data.table)
library(dplyr)
library(fixest)
library(ggplot2)


# Getting the workign directory 
shell_root <- "/scratch/gpfs/ar8787/groupdata2/india_forest_land" 
dbox_root <- "~/Dropbox/india_forest_land" 
root <- shell_root
setwd( root )

### Geting State names
state_names <- fread( "A_MicroData/state_names_shrug.csv" )
state_id_code <- {int(n_sample)}
st_name <- state_names[ state_names$state_id == state_id_code ]$state_name


# Getting the num of files
df1 <- fread("A_MicroData/data_sysdif.csv")

# Add a new column 'is_duplicate' which is TRUE for duplicated rows based on 'vill_id' and 'year'
df1[, is_duplicate := duplicated(.SD) | duplicated(.SD, fromLast = TRUE), .SDcols = c('vill_id', 'year')]
df1[ df1$is_duplicate == 0 ]


# Get year of treatment
# Get the minimum year where ps_ror_data_entry equals 1 for each vill_id
min_yr_tr <- df1[post_ror_data_entry == 1, .(min_year = min(year)), by = .(vill_id)]
# Merge with the original data.table by vill_id
df1 <- merge(df1, min_yr_tr, by = "vill_id", all.x = TRUE)
df1[is.na(min_year), min_year := 0]

df1 <- df1[df1$pc11_state_id == state_id_code ]
df_year_2010 <- df1[df1$year == 2010]


# Calculate the median of the 'value' column
median_value <- median(df_year_2010$per_treecover)
# Generate a new variable 'above_median'; it will be TRUE if 'value' is greater than the median
df_year_2010[, above_median_all := (per_treecover > median_value)*1 ]
df_year_2010 <- df_year_2010[, .(shrid, above_median_all )]

df2 <- df1 %>% left_join( df_year_2010 )
df2_above <- df2[df2$above_median_all == 1]
df2_below <- df2[df2$above_median_all == 0]

#-------------------------------------------------------------------------------



#-------------------------------------------------------------------------------
# above the median
#-------------------------------------------------------------------------------

# Start estimation
out <- att_gt(yname = "per_treecover",
              gname = "min_year",
              idname = "vill_id",
              tname = "year",
              xformla = ~ 1 ,
              data = df2_above,
              est_method = "reg"
)

filename <- paste0( "above_median_", state_id_code, ".RDS")
path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                  filename)
saveRDS( out, path )


# aggregate the group-time average treatment effects
dynamic_sum <- aggte( out, type = "dynamic")
res <- ggdid(dynamic_sum)
data_sum <- res$data
data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
est_df <- data.table( data_sum )

# Vectors of old and new column names
oldnames <- c( "year", "att", "ymax", "ymin" )
newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
setnames(est_df, oldnames, newnames)

# Getting the number of obseravtions
n_df <- df2_above[t2ev_ror_data_entry %in% est_df$Time][, .N, by = t2ev_ror_data_entry]
n_df$Time <- n_df$t2ev_ror_data_entry
setorder(n_df,Time )
setorder(est_df,Time )


# Getting factors
rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                  min(est_df[,"Estimate"], na.rm = TRUE))/2
scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95

est_df[,"xmin"] <- est_df[,"Time"] - 0.2
est_df[,"xmax"] <- est_df[,"Time"] + 0.2
est_df[,"ymin"] <- min_y_lim
est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )


# Getting the plot
p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
  geom_line(color = "blue") +
  geom_point(color = "red") +
  geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                width = 0.2, color = "red") +
  geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                               ymax = ymax), 
            fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
  coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
  scale_y_continuous(name = "Estimate", 
                     sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                         name = "Number of Observations")) + 
  labs(y = "ATT", x = "Time to First Year of Treatment", 
       title = "Ror Data Entry at Level") + 
  geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
  geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
  ggtitle( paste0( st_name, " Non yet Treated and Never Treated" ) ) +
  theme_minimal()

filename <- paste0( "above_median_", state_id_code, "_plot.png")
path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                  filename)
ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

#-------------------------------------------------------------------------------





#-------------------------------------------------------------------------------
# below the median
#-------------------------------------------------------------------------------

# Start estimation
out <- att_gt(yname = "per_treecover",
              gname = "min_year",
              idname = "vill_id",
              tname = "year",
              xformla = ~ 1 ,
              data = df2_below,
              est_method = "reg"
)

filename <- paste0( "below_median_", state_id_code, ".RDS")
path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                  filename)
saveRDS( out, path )


# aggregate the group-time average treatment effects
dynamic_sum <- aggte( out, type = "dynamic")
res <- ggdid(dynamic_sum)
data_sum <- res$data
data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
est_df <- data.table( data_sum )

# Vectors of old and new column names
oldnames <- c( "year", "att", "ymax", "ymin" )
newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
setnames(est_df, oldnames, newnames)

# Getting the number of obseravtions
n_df <- df2_below[t2ev_ror_data_entry %in% est_df$Time][, .N, by = t2ev_ror_data_entry]
n_df$Time <- n_df$t2ev_ror_data_entry
setorder(n_df,Time )
setorder(est_df,Time )


# Getting factors
rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                  min(est_df[,"Estimate"], na.rm = TRUE))/2
scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95

est_df[,"xmin"] <- est_df[,"Time"] - 0.2
est_df[,"xmax"] <- est_df[,"Time"] + 0.2
est_df[,"ymin"] <- min_y_lim
est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )


# Getting the plot
p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
  geom_line(color = "blue") +
  geom_point(color = "red") +
  geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                width = 0.2, color = "red") +
  geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                               ymax = ymax), 
            fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
  coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
  scale_y_continuous(name = "Estimate", 
                     sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                         name = "Number of Observations")) + 
  labs(y = "ATT", x = "Time to First Year of Treatment", 
       title = "Ror Data Entry at Level") + 
  geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
  geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
  ggtitle( paste0( st_name, " Non yet Treated and Never Treated" ) ) +
  theme_minimal()

filename <- paste0( "below_median_", state_id_code, "_plot.png")
path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                  filename)
ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

#-------------------------------------------------------------------------------

"""
    
    # Saving R scripts
    final_code = text1 
    filename = f"sate_{n_sample}_vill"
    with open( fr"{main_path}/batch/{filename}.R", "w") as f:
        f.write( f"{final_code}")
    
    
    text = f"""#!/bin/bash
#SBATCH --nodes=1                # node count
#SBATCH --ntasks=1               # total number of tasks across all nodes
#SBATCH --cpus-per-task=1        # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --mem-per-cpu=10G         # memory per cpu-core (4G per cpu-core is default)
#SBATCH --time=08:00:00          # total run time limit (HH:MM:SS)
#SBATCH --mail-user=futurolos9@gmail.com
#SBATCH --mail-type=BEGIN
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL
#SBATCH --output={filename}.log

module purge
R CMD BATCH {main_path}/batch/{filename}.R
"""
    
    with open( f"{main_path}/batch/{filename}.sbatch", "w") as f:
            f.write( f"{text}")

    n_sample = n_sample + 1

In [32]:
states_sel = [34, 35, 25, 32, 31, 30, 18, 22, 13 , 12, 17, 26, 27 , 8, 5, 1]
for i in states_sel:

    text1 = f"""

#-------------------------------------------------------------------------------

rm( list = ls() )
library(did)
library(augsynth)
library(data.table)
library(dplyr)
library(fixest)
library(ggplot2)


# Getting the workign directory 
shell_root <- "/scratch/gpfs/ar8787/groupdata2/india_forest_land" 
dbox_root <- "~/Dropbox/india_forest_land" 
root <- shell_root
setwd( root )

### Geting State names
state_names <- fread( "A_MicroData/state_names_shrug.csv" )
state_id_code <- {int(i)}
st_name <- state_names[ state_names$state_id == state_id_code ]$state_name


# Getting the num of files
df1 <- fread("A_MicroData/data_sysdif.csv")

# Add a new column 'is_duplicate' which is TRUE for duplicated rows based on 'vill_id' and 'year'
df1[, is_duplicate := duplicated(.SD) | duplicated(.SD, fromLast = TRUE), .SDcols = c('vill_id', 'year')]
df1[ df1$is_duplicate == 0 ]


# Get year of treatment
# Get the minimum year where ps_ror_data_entry equals 1 for each vill_id
min_yr_tr <- df1[post_ror_data_entry == 1, .(min_year = min(year)), by = .(vill_id)]
# Merge with the original data.table by vill_id
df1 <- merge(df1, min_yr_tr, by = "vill_id", all.x = TRUE)
df1[is.na(min_year), min_year := 0]

df1 <- df1[df1$pc11_state_id == state_id_code ]
df_year_2010 <- df1[df1$year == 2010]


# Calculate the median of the 'value' column
median_value <- median(df_year_2010$per_treecover)
# Generate a new variable 'above_median'; it will be TRUE if 'value' is greater than the median
df_year_2010[, above_median_all := (per_treecover > median_value)*1 ]
df_year_2010 <- df_year_2010[, .(shrid, above_median_all )]

df2 <- df1 %>% left_join( df_year_2010 )
df2_above <- df2[df2$above_median_all == 1]
df2_below <- df2[df2$above_median_all == 0]

#-------------------------------------------------------------------------------



#-------------------------------------------------------------------------------
# above the median
#-------------------------------------------------------------------------------

# Start estimation
out <- att_gt(yname = "per_treecover",
              gname = "min_year",
              idname = "vill_id",
              tname = "year",
              xformla = ~ 1 ,
              data = df2_above,
              est_method = "reg", 
              control_group=c( "notyettreated" )
)

filename <- paste0( "above_median_", state_id_code, ".RDS")
path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                  filename)
saveRDS( out, path )


# aggregate the group-time average treatment effects
dynamic_sum <- aggte( out, type = "dynamic")
res <- ggdid(dynamic_sum)
data_sum <- res$data
data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
est_df <- data.table( data_sum )

# Vectors of old and new column names
oldnames <- c( "year", "att", "ymax", "ymin" )
newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
setnames(est_df, oldnames, newnames)

# Getting the number of obseravtions
n_df <- df2_above[t2ev_ror_data_entry %in% est_df$Time][, .N, by = t2ev_ror_data_entry]
n_df$Time <- n_df$t2ev_ror_data_entry
setorder(n_df,Time )
setorder(est_df,Time )


# Getting factors
rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                  min(est_df[,"Estimate"], na.rm = TRUE))/2
scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95

est_df[,"xmin"] <- est_df[,"Time"] - 0.2
est_df[,"xmax"] <- est_df[,"Time"] + 0.2
est_df[,"ymin"] <- min_y_lim
est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )


# Getting the plot
p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
  geom_line(color = "blue") +
  geom_point(color = "red") +
  geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                width = 0.2, color = "red") +
  geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                               ymax = ymax), 
            fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
  coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
  scale_y_continuous(name = "Estimate", 
                     sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                         name = "Number of Observations")) + 
  labs(y = "ATT", x = "Time to First Year of Treatment", 
       title = "Ror Data Entry at Level") + 
  geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
  geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
  ggtitle( paste0( st_name, " Non yet Treated" ) ) +
  theme_minimal()

filename <- paste0( "above_median_", state_id_code, "_plot.png")
path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                  filename)
ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

#-------------------------------------------------------------------------------





#-------------------------------------------------------------------------------
# below the median
#-------------------------------------------------------------------------------

# Start estimation
out <- att_gt(yname = "per_treecover",
              gname = "min_year",
              idname = "vill_id",
              tname = "year",
              xformla = ~ 1 ,
              data = df2_below,
              est_method = "reg", 
              control_group=c( "notyettreated" )
              
)

filename <- paste0( "below_median_", state_id_code, ".RDS")
path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                  filename)
saveRDS( out, path )


# aggregate the group-time average treatment effects
dynamic_sum <- aggte( out, type = "dynamic")
res <- ggdid(dynamic_sum)
data_sum <- res$data
data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
est_df <- data.table( data_sum )

# Vectors of old and new column names
oldnames <- c( "year", "att", "ymax", "ymin" )
newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
setnames(est_df, oldnames, newnames)

# Getting the number of obseravtions
n_df <- df2_below[t2ev_ror_data_entry %in% est_df$Time][, .N, by = t2ev_ror_data_entry]
n_df$Time <- n_df$t2ev_ror_data_entry
setorder(n_df,Time )
setorder(est_df,Time )


# Getting factors
rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                  min(est_df[,"Estimate"], na.rm = TRUE))/2
scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95

est_df[,"xmin"] <- est_df[,"Time"] - 0.2
est_df[,"xmax"] <- est_df[,"Time"] + 0.2
est_df[,"ymin"] <- min_y_lim
est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )


# Getting the plot
p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
  geom_line(color = "blue") +
  geom_point(color = "red") +
  geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                width = 0.2, color = "red") +
  geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                               ymax = ymax), 
            fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
  coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
  scale_y_continuous(name = "Estimate", 
                     sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                         name = "Number of Observations")) + 
  labs(y = "ATT", x = "Time to First Year of Treatment", 
       title = "Ror Data Entry at Level") + 
  geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
  geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
  ggtitle( paste0( st_name, " Non yet Treated" ) ) +
  theme_minimal()

filename <- paste0( "below_median_", state_id_code, "_plot.png")
path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                  filename)
ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

#-------------------------------------------------------------------------------

"""
    
    # Saving R scripts
    final_code = text1 
    filename = f"sate_{i}_vill"
    with open( fr"{main_path}/batch/{filename}.R", "w") as f:
        f.write( f"{final_code}")
    
    
    text = f"""#!/bin/bash
#SBATCH --nodes=1                # node count
#SBATCH --ntasks=1               # total number of tasks across all nodes
#SBATCH --cpus-per-task=1        # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --mem-per-cpu=10G         # memory per cpu-core (4G per cpu-core is default)
#SBATCH --time=08:00:00          # total run time limit (HH:MM:SS)
#SBATCH --mail-user=futurolos9@gmail.com
#SBATCH --mail-type=BEGIN
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL
#SBATCH --output={filename}.log

module purge
R CMD BATCH {main_path}/batch/{filename}.R
"""
    
    with open( f"{main_path}/batch/{filename}.sbatch", "w") as f:
            f.write( f"{text}")

In [17]:
n1 = n_sample - 1
files = np.arange(1, n_sample).reshape( 1, n1 )
names = ['anzony' ]

text1 = f"""
# submit
cd {main_path}/batch
"""
i = 0
for name in names:
    vlaset = files[ i, :]
    
    text2 = ""
    for val in vlaset:

        if val < n_sample:
            text2 = text2 + f"""
sbatch sate_{val}_vill.sbatch
                    """
            
    with open( fr"{main_path}/anzony_vill.sh", "w") as f:
        f.write( f"{text1 + text2}")
    i = i + 1


In [30]:
n1 = n_sample - 1
files = np.arange(1, n_sample).reshape( 1, n1 )
names = ['anzony' ]

text1 = f"""
# submit
cd {main_path}/batch
"""
i = 0
for name in names:
    vlaset = files[ i, :]
    
    text2 = ""
    for val in states_sel:

        if val < n_sample:
            text2 = text2 + f"""
sbatch sate_{val}_vill.sbatch
                    """
            
    with open( fr"{main_path}/anzony_vill2.sh", "w") as f:
        f.write( f"{text1 + text2}")
    i = i + 1


## Block 50


In [88]:
# Main paths
main_path = "/scratch/gpfs/ar8787/groupdata2/india_forest_land/C_Programs/_vcf_10_callaway_santanna_analysis/median_filter/by_state"
root = Path(main_path)

In [89]:
total_files = 35 

In [90]:
n_sample = 1
for i in np.arange(0, total_files):

    text1 = f"""
rm(list = ls())
library(augsynth)
library(data.table)
library(dplyr)
library(fixest)
library(ggplot2)
library(did)


# Getting the workign directory 
shell_root <- "/scratch/gpfs/ar8787/groupdata2/india_forest_land" 
dbox_root <- "~/Dropbox/india_forest_land" 
root <- shell_root
setwd( root )

### Geting State names
state_names <- fread( "A_MicroData/state_names_shrug.csv" )
state_id_code <- {int(i) + 1 }
st_name <- state_names[ state_names$state_id == state_id_code ]$state_name


# Getting the num of files
# Import data
df1 <- fread( "A_MicroData/data_sysdif_block_lvl.csv" )
names(df1)
df1 <- df1[df1$state_id == state_id_code ]
df_year_2010 <- df1[df1$year == 2010]


# Calculate the median of the 'value' column
median_value <- median(df_year_2010$per_treecover)
# Generate a new variable 'above_median'; it will be TRUE if 'value' is greater than the median
df_year_2010[, above_median_block_50_all := (per_treecover > median_value)*1 ]
df_year_2010 <- df_year_2010[, .(block_id, above_median_block_50_all )]

df2 <- df1 %>% left_join( df_year_2010 )

# Generation of t2event
# Create a data.table with minimum year by block_id
min_years_data <- df2[
  post_ror_data_entry_block_50 == 1,
  .(min_year = min(year)), # Replace 'year_column' with your actual year column name
  by = .(block_id)
]



# Merge the min_years_data back to the original dataset
df2 <- merge(df2, min_years_data, by = "block_id", all.x = TRUE)
df2[is.na(min_year), min_year := 0]
df2$t2ev_ror_data_entry_block_50 <- df2$year - df2$min_year


df2_above <- df2[df2$above_median_block_50_all == 1]
df2_below <- df2[df2$above_median_block_50_all == 0]



#-------------------------------------------------------------------------------
# above the median
#-------------------------------------------------------------------------------

len_above = length(unique(df2_above$min_year))
"""
    text2 = """
if ( len_above > 1 ){
    
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_above,
                est_method = "reg"
  )
  
  filename <- paste0( "above_median_block_50_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_above[t2ev_ror_data_entry_block_50 %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_50]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_50
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, "- Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "above_median_block_50_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

}
#-------------------------------------------------------------------------------





#-------------------------------------------------------------------------------
# below the median
#-------------------------------------------------------------------------------

len_below = length(unique(df2_below$min_year))

if ( len_below > 1 ){
  
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_below,
                est_method = "reg"
  )
  
  filename <- paste0( "below_median_block_50_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_below[t2ev_ror_data_entry_block_50 %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_50]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_50
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, " - Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "below_median_block_50_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

}
#-------------------------------------------------------------------------------

"""
    
    # Saving R scripts
    final_code = text1 + text2
    filename = f"sate_{n_sample}_block_50"
    with open( fr"{main_path}/batch/{filename}.R", "w") as f:
        f.write( f"{final_code}")
    
    
    text = f"""#!/bin/bash
#SBATCH --nodes=1                # node count
#SBATCH --ntasks=1               # total number of tasks across all nodes
#SBATCH --cpus-per-task=1        # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --mem-per-cpu=4G         # memory per cpu-core (4G per cpu-core is default)
#SBATCH --time=08:00:00          # total run time limit (HH:MM:SS)
#SBATCH --mail-user=futurolos9@gmail.com
#SBATCH --mail-type=BEGIN
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL
#SBATCH --output={filename}.log

module purge
R CMD BATCH {main_path}/batch/{filename}.R
"""
    
    with open( f"{main_path}/batch/{filename}.sbatch", "w") as f:
            f.write( f"{text}")

    n_sample = n_sample + 1

In [93]:
n1 = n_sample - 1
files = np.arange(1, n_sample).reshape( 1, n1 )
names = ['anzony' ]

text1 = f"""
# submit
cd {main_path}/batch
"""
i = 0
for name in names:
    vlaset = files[ i, :]
    
    text2 = ""
    for val in vlaset:

        if val < n_sample:
            text2 = text2 + f"""
sbatch sate_{val}_block_50.sbatch
                    """
            
    with open( fr"{main_path}/len_below.sh", "w") as f:
        f.write( f"{text1 + text2}")
    i = i + 1


In [92]:
states_sel = [34, 35, 25, 32, 31, 30, 18, 22, 13 , 12, 17, 26, 27 , 8, 5, 1]
for i in states_sel:
    text1 = f"""
rm(list = ls())
library(augsynth)
library(data.table)
library(dplyr)
library(fixest)
library(ggplot2)
library(did)


# Getting the workign directory 
shell_root <- "/scratch/gpfs/ar8787/groupdata2/india_forest_land" 
dbox_root <- "~/Dropbox/india_forest_land" 
root <- shell_root
setwd( root )

### Geting State names
state_names <- fread( "A_MicroData/state_names_shrug.csv" )
state_id_code <- {int(i)}
st_name <- state_names[ state_names$state_id == state_id_code ]$state_name


# Getting the num of files
# Import data
df1 <- fread( "A_MicroData/data_sysdif_block_lvl.csv" )
names(df1)
df1 <- df1[df1$state_id == state_id_code ]
df_year_2010 <- df1[df1$year == 2010]


# Calculate the median of the 'value' column
median_value <- median(df_year_2010$per_treecover)
# Generate a new variable 'above_median'; it will be TRUE if 'value' is greater than the median
df_year_2010[, above_median_block_50_all := (per_treecover > median_value)*1 ]
df_year_2010 <- df_year_2010[, .(block_id, above_median_block_50_all )]

df2 <- df1 %>% left_join( df_year_2010 )

# Generation of t2event
# Create a data.table with minimum year by block_id
min_years_data <- df2[
  post_ror_data_entry_block_50 == 1,
  .(min_year = min(year)), # Replace 'year_column' with your actual year column name
  by = .(block_id)
]



# Merge the min_years_data back to the original dataset
df2 <- merge(df2, min_years_data, by = "block_id", all.x = TRUE)
df2[is.na(min_year), min_year := 0]
df2$t2ev_ror_data_entry_block_50 <- df2$year - df2$min_year


df2_above <- df2[df2$above_median_block_50_all == 1]
df2_below <- df2[df2$above_median_block_50_all == 0]



#-------------------------------------------------------------------------------
# above the median
#-------------------------------------------------------------------------------

len_above = length(unique(df2_above$min_year))
len_above
"""
    text2 = """
if ( len_above > 1 ){
    
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_above,
                est_method = "reg",
                control_group = "notyettreated"
  )
  
  filename <- paste0( "above_median_block_50_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum = na.omit(data_sum)
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_above[t2ev_ror_data_entry_block_50 %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_50]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_50
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, "- Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "above_median_block_50_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

}
#-------------------------------------------------------------------------------





#-------------------------------------------------------------------------------
# below the median
#-------------------------------------------------------------------------------

len_below = length(unique(df2_below$min_year))
len_below
if ( len_below > 1 ){
  
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_below,
                est_method = "reg",
                control_group = "notyettreated"
  )
  
  filename <- paste0( "below_median_block_50_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum = na.omit(data_sum)
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_below[t2ev_ror_data_entry_block_50 %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_50]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_50
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, " - Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "below_median_block_50_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

}
#-------------------------------------------------------------------------------

"""
    
    # Saving R scripts
    final_code = text1 + text2
    filename = f"sate_{i}_block_50"
    with open( fr"{main_path}/batch/{filename}.R", "w") as f:
        f.write( f"{final_code}")
    
    
    text = f"""#!/bin/bash
#SBATCH --nodes=1                # node count
#SBATCH --ntasks=1               # total number of tasks across all nodes
#SBATCH --cpus-per-task=1        # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --mem-per-cpu=4G         # memory per cpu-core (4G per cpu-core is default)
#SBATCH --time=08:00:00          # total run time limit (HH:MM:SS)
#SBATCH --mail-user=futurolos9@gmail.com
#SBATCH --mail-type=BEGIN
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL
#SBATCH --output={filename}.log

module purge
R CMD BATCH {main_path}/batch/{filename}.R
"""
    
    with open( f"{main_path}/batch/{filename}.sbatch", "w") as f:
            f.write( f"{text}")


In [59]:
n1 = n_sample - 1
files = np.arange(1, n_sample).reshape( 1, n1 )
names = ['anzony' ]

text1 = f"""
# submit
cd {main_path}/batch
"""
i = 0
for name in names:
    vlaset = files[ i, :]
    
    text2 = ""
    for val in vlaset:

        if val < n_sample:
            text2 = text2 + f"""
sbatch sate_{val}_block_50.sbatch
                    """
            
    with open( fr"{main_path}/anzony_block_50_2.sh", "w") as f:
        f.write( f"{text1 + text2}")
    i = i + 1


In [94]:
states_sel = [ 33, 28, 23, 21, 20, 19, 16, 6, 3, 2, 34, 29, 22, 12 ]
for i in states_sel:
    text1 = f"""
rm(list = ls())
library(augsynth)
library(data.table)
library(dplyr)
library(fixest)
library(ggplot2)
library(did)


# Getting the workign directory 
shell_root <- "/scratch/gpfs/ar8787/groupdata2/india_forest_land" 
dbox_root <- "~/Dropbox/india_forest_land" 
root <- shell_root
setwd( root )

### Geting State names
state_names <- fread( "A_MicroData/state_names_shrug.csv" )
state_id_code <- {int(i)}
st_name <- state_names[ state_names$state_id == state_id_code ]$state_name


# Getting the num of files
# Import data
df1 <- fread( "A_MicroData/data_sysdif_block_lvl.csv" )
names(df1)
df1 <- df1[df1$state_id == state_id_code ]
df_year_2010 <- df1[df1$year == 2010]


# Calculate the median of the 'value' column
median_value <- median(df_year_2010$per_treecover)
# Generate a new variable 'above_median'; it will be TRUE if 'value' is greater than the median
df_year_2010[, above_median_block_50_all := (per_treecover > median_value)*1 ]
df_year_2010 <- df_year_2010[, .(block_id, above_median_block_50_all )]

df2 <- df1 %>% left_join( df_year_2010 )

# Generation of t2event
# Create a data.table with minimum year by block_id
min_years_data <- df2[
  post_ror_data_entry_block_50 == 1,
  .(min_year = min(year)), # Replace 'year_column' with your actual year column name
  by = .(block_id)
]



# Merge the min_years_data back to the original dataset
df2 <- merge(df2, min_years_data, by = "block_id", all.x = TRUE)
df2[is.na(min_year), min_year := 0]
df2$t2ev_ror_data_entry_block_50 <- df2$year - df2$min_year


df2_above <- df2[df2$above_median_block_50_all == 1]
df2_below <- df2[df2$above_median_block_50_all == 0]



#-------------------------------------------------------------------------------
# above the median
#-------------------------------------------------------------------------------

len_above = length(unique(df2_above$min_year))
len_above
"""
    text2 = """
if ( len_above > 1 ){
    
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_above,
                est_method = "reg",
                control_group = "notyettreated"
  )
  
  filename <- paste0( "above_median_block_50_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum = na.omit(data_sum)
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_above[t2ev_ror_data_entry_block_50 %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_50]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_50
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, "- Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "above_median_block_50_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

}
#-------------------------------------------------------------------------------





#-------------------------------------------------------------------------------
# below the median
#-------------------------------------------------------------------------------

len_below = length(unique(df2_below$min_year))
len_below
if ( len_below > 1 ){
  
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_below,
                est_method = "reg",
                control_group = "notyettreated"
  )
  
  filename <- paste0( "below_median_block_50_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum = na.omit(data_sum)
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_below[t2ev_ror_data_entry_block_50 %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_50]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_50
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, " - Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "below_median_block_50_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

}
#-------------------------------------------------------------------------------

"""
    
    # Saving R scripts
    final_code = text1 + text2
    filename = f"sate_{i}_block_50"
    with open( fr"{main_path}/batch/{filename}.R", "w") as f:
        f.write( f"{final_code}")
    
    
    text = f"""#!/bin/bash
#SBATCH --nodes=1                # node count
#SBATCH --ntasks=1               # total number of tasks across all nodes
#SBATCH --cpus-per-task=1        # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --mem-per-cpu=4G         # memory per cpu-core (4G per cpu-core is default)
#SBATCH --time=08:00:00          # total run time limit (HH:MM:SS)
#SBATCH --mail-user=futurolos9@gmail.com
#SBATCH --mail-type=BEGIN
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL
#SBATCH --output={filename}.log

module purge
R CMD BATCH {main_path}/batch/{filename}.R
"""
    
    with open( f"{main_path}/batch/{filename}.sbatch", "w") as f:
            f.write( f"{text}")


In [95]:
n1 = n_sample - 1
files = np.arange(1, n_sample).reshape( 1, n1 )
names = ['anzony' ]

text1 = f"""
# submit
cd {main_path}/batch
"""
i = 0
for name in names:
    vlaset = files[ i, :]
    
    text2 = ""
    for val in states_sel:

        if val < n_sample:
            text2 = text2 + f"""
sbatch sate_{val}_block_50.sbatch
                    """
            
    with open( fr"{main_path}/anzony_block_50_3.sh", "w") as f:
        f.write( f"{text1 + text2}")
    i = i + 1


In [97]:
import glob
import re

# Directory path
directory_path = "/scratch/gpfs/ar8787/groupdata2/india_forest_land/F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state"

# Pattern to match files that start with "above_median_block_any"
file_pattern = "above_median_block_50*"

# Use glob to find files matching the pattern
matching_files = glob.glob(f"{directory_path}/{file_pattern}")

# Regular expression pattern to extract numbers from file names
number_pattern = r'\d+'

# Initialize a list to store extracted numbers
numbers_in_file_names = []

# Iterate through matching files and extract numbers
for file in matching_files:
    match = re.search(number_pattern, file.split("/")[-1].replace("_50", "") )
    if match:
        numbers_in_file_names.append(int(match.group()))

# Print the extracted numbers
for number in numbers_in_file_names:
    print(number)


21
13
16
2
15
34
20
19
23
6
3
27
33
22
8
5
30
18
10
32
28


In [99]:
list_states = np.arange(1, total_files+1)

In [103]:
list_sel = set(list_states).difference( set(numbers_in_file_names) )

In [104]:
for i in list_sel:
    text1 = f"""
rm(list = ls())
library(augsynth)
library(data.table)
library(dplyr)
library(fixest)
library(ggplot2)
library(did)


# Getting the workign directory 
shell_root <- "/scratch/gpfs/ar8787/groupdata2/india_forest_land" 
dbox_root <- "~/Dropbox/india_forest_land" 
root <- shell_root
setwd( root )

### Geting State names
state_names <- fread( "A_MicroData/state_names_shrug.csv" )
state_id_code <- {int(i)}
st_name <- state_names[ state_names$state_id == state_id_code ]$state_name


# Getting the num of files
# Import data
df1 <- fread( "A_MicroData/data_sysdif_block_lvl.csv" )
names(df1)
df1 <- df1[df1$state_id == state_id_code ]
df_year_2010 <- df1[df1$year == 2010]


# Calculate the median of the 'value' column
median_value <- median(df_year_2010$per_treecover)
# Generate a new variable 'above_median'; it will be TRUE if 'value' is greater than the median
df_year_2010[, above_median_block_50_all := (per_treecover > median_value)*1 ]
df_year_2010 <- df_year_2010[, .(block_id, above_median_block_50_all )]

df2 <- df1 %>% left_join( df_year_2010 )

# Generation of t2event
# Create a data.table with minimum year by block_id
min_years_data <- df2[
  post_ror_data_entry_block_50 == 1,
  .(min_year = min(year)), # Replace 'year_column' with your actual year column name
  by = .(block_id)
]



# Merge the min_years_data back to the original dataset
df2 <- merge(df2, min_years_data, by = "block_id", all.x = TRUE)
df2[is.na(min_year), min_year := 0]
df2$t2ev_ror_data_entry_block_50 <- df2$year - df2$min_year


df2_above <- df2[df2$above_median_block_50_all == 1]
df2_below <- df2[df2$above_median_block_50_all == 0]



#-------------------------------------------------------------------------------
# above the median
#-------------------------------------------------------------------------------

len_above = length(unique(df2_above$min_year))
len_above
"""
    text2 = """
if ( len_above > 1 ){
    
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_above,
                est_method = "reg",
                control_group = "notyettreated"
  )
  
  filename <- paste0( "above_median_block_50_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum = na.omit(data_sum)
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_above[t2ev_ror_data_entry_block_50 %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_50]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_50
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, "- Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "above_median_block_50_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

}
#-------------------------------------------------------------------------------





#-------------------------------------------------------------------------------
# below the median
#-------------------------------------------------------------------------------

len_below = length(unique(df2_below$min_year))
len_below
if ( len_below > 1 ){
  
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_below,
                est_method = "reg",
                control_group = "notyettreated"
  )
  
  filename <- paste0( "below_median_block_50_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum = na.omit(data_sum)
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_below[t2ev_ror_data_entry_block_50 %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_50]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_50
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, " - Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "below_median_block_50_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)

}
#-------------------------------------------------------------------------------

"""
    
    # Saving R scripts
    final_code = text1 + text2
    filename = f"sate_{i}_block_50"
    with open( fr"{main_path}/batch/{filename}.R", "w") as f:
        f.write( f"{final_code}")
    
    
    text = f"""#!/bin/bash
#SBATCH --nodes=1                # node count
#SBATCH --ntasks=1               # total number of tasks across all nodes
#SBATCH --cpus-per-task=1        # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --mem-per-cpu=4G         # memory per cpu-core (4G per cpu-core is default)
#SBATCH --time=08:00:00          # total run time limit (HH:MM:SS)
#SBATCH --mail-user=futurolos9@gmail.com
#SBATCH --mail-type=BEGIN
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL
#SBATCH --output={filename}.log

module purge
R CMD BATCH {main_path}/batch/{filename}.R
"""
    
    with open( f"{main_path}/batch/{filename}.sbatch", "w") as f:
            f.write( f"{text}")


In [106]:
n1 = n_sample - 1
files = np.arange(1, n_sample).reshape( 1, n1 )
names = ['anzony' ]

text1 = f"""
# submit
cd {main_path}/batch
"""
i = 0
for name in names:
    vlaset = files[ i, :]
    
    text2 = ""
    for val in list_sel:

        if val < n_sample:
            text2 = text2 + f"""
sbatch sate_{val}_block_50.sbatch
                    """
            
    with open( fr"{main_path}/anzony_block_50_4.sh", "w") as f:
        f.write( f"{text1 + text2}")
    i = i + 1


In [105]:
list_sel

{1, 4, 7, 9, 11, 12, 14, 17, 24, 25, 26, 29, 31, 35}

## Block Any

In [45]:
n_sample

36

In [60]:
n_sample = 1
for i in np.arange(0, total_files):

    text1 = f"""
rm(list = ls())
library(augsynth)
library(data.table)
library(dplyr)
library(fixest)
library(ggplot2)
library(did)


# Getting the workign directory 
shell_root <- "/scratch/gpfs/ar8787/groupdata2/india_forest_land" 
dbox_root <- "~/Dropbox/india_forest_land" 
root <- shell_root
setwd( root )

### Geting State names
state_names <- fread( "A_MicroData/state_names_shrug.csv" )
state_id_code <- {int(i)+1}
st_name <- state_names[ state_names$state_id == state_id_code ]$state_name


# Getting the num of files
# Import data
df1 <- fread( "A_MicroData/data_sysdif_block_lvl.csv" )
names(df1)

df1 <- df1[df1$state_id == state_id_code ]
df_year_2010 <- df1[df1$year == 2010]


# Calculate the median of the 'value' column
median_value <- median(df_year_2010$per_treecover)
# Generate a new variable 'above_median'; it will be TRUE if 'value' is greater than the median
df_year_2010[, above_median_block_any_all := (per_treecover > median_value)*1 ]
df_year_2010 <- df_year_2010[, .(block_id, above_median_block_any_all )]

df2 <- df1 %>% left_join( df_year_2010 )

# Generation of t2event
# Create a data.table with minimum year by block_id
min_years_data <- df2[
  post_ror_data_entry_block_any == 1,
  .(min_year = min(year)), # Replace 'year_column' with your actual year column name
  by = .(block_id)
]



# Merge the min_years_data back to the original dataset
df2 <- merge(df2, min_years_data, by = "block_id", all.x = TRUE)
df2[is.na(min_year), min_year := 0]
df2$t2ev_ror_data_entry_block_any <- df2$year - df2$min_year


df2_above <- df2[df2$above_median_block_any_all == 1]
df2_below <- df2[df2$above_median_block_any_all == 0]
"""
    text2 = """


#-------------------------------------------------------------------------------
# above the median
#-------------------------------------------------------------------------------

len_above = length(unique(df2_above$min_year))

if ( len_above > 1 ){
  
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_above,
                est_method = "reg"
  )
  
  filename <- paste0( "above_median_block_any_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_above[t2ev_ror_data_entry_block_any %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_any]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_any
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, "- Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "above_median_block_any_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)
  
}
#-------------------------------------------------------------------------------





#-------------------------------------------------------------------------------
# below the median
#-------------------------------------------------------------------------------

len_below = length(unique(df2_below$min_year))

if ( len_below > 1 ){
  
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_below,
                est_method = "reg"
  )
  
  filename <- paste0( "below_median_block_any_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_below[t2ev_ror_data_entry_block_any %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_any]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_any
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, " - Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "below_median_block_any_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)
  
}
#-------------------------------------------------------------------------------
"""
    
    # Saving R scripts
    final_code = text1 + text2
    filename = f"sate_{int(i+1)}_block_any"
    with open( fr"{main_path}/batch/{filename}.R", "w") as f:
        f.write( f"{final_code}")
    
    
    text = f"""#!/bin/bash
#SBATCH --nodes=1                # node count
#SBATCH --ntasks=1               # total number of tasks across all nodes
#SBATCH --cpus-per-task=1        # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --mem-per-cpu=4G         # memory per cpu-core (4G per cpu-core is default)
#SBATCH --time=08:00:00          # total run time limit (HH:MM:SS)
#SBATCH --mail-user=futurolos9@gmail.com
#SBATCH --mail-type=BEGIN
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL
#SBATCH --output={filename}.log

module purge
R CMD BATCH {main_path}/batch/{filename}.R
"""
    
    with open( f"{main_path}/batch/{filename}.sbatch", "w") as f:
            f.write( f"{text}")

    n_sample = n_sample + 1

In [61]:
n1 = n_sample - 1
files = np.arange(1, n_sample).reshape( 1, n1 )
names = ['anzony' ]

text1 = f"""
# submit
cd {main_path}/batch
"""
i = 0
for name in names:
    vlaset = files[ i, :]
    
    text2 = ""
    for val in vlaset:

        if val < n_sample:
            text2 = text2 + f"""
sbatch sate_{val}_block_any.sbatch
                    """
            
    with open( fr"{main_path}/anzony_block_any.sh", "w") as f:
        f.write( f"{text1 + text2}")
    i = i + 1


In [84]:
list_state = np.arange(1, total_files+1)

In [85]:
all_files = os.listdir("/scratch/gpfs/ar8787/groupdata2/india_forest_land/F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state")

In [ ]:
import glob
import re

# Directory path
directory_path = "/scratch/gpfs/ar8787/groupdata2/india_forest_land/F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state"

# Pattern to match files that start with "above_median_block_any"
file_pattern = "above_median_block_any*"

# Use glob to find files matching the pattern
matching_files = glob.glob(f"{directory_path}/{file_pattern}")

# Regular expression pattern to extract numbers from file names
number_pattern = r'\d+'

# Initialize a list to store extracted numbers
numbers_in_file_names = []

# Iterate through matching files and extract numbers
for file in matching_files:
    match = re.search(number_pattern, file.split("/")[-1])
    if match:
        numbers_in_file_names.append(int(match.group()))

# Print the extracted numbers
for number in numbers_in_file_names:
    print(number)


In [81]:
states_sel = list(set(list_state).difference(set(numbers_in_file_names)))
states_sel

[32, 33, 35, 4, 7, 12, 17, 19, 20, 21, 22, 23, 24, 25, 26, 29, 31]

In [82]:
states_sel = [32, 33, 35, 4, 7, 12, 17, 19, 20, 21, 22, 23, 24, 25, 26, 29, 31,
              11, 10, 8, 5, 6, 2, 3, 34, 35, 24, 28, 26, 27, 30, 16, 14 ]
for i in states_sel:
    text1 = f"""
rm(list = ls())
library(augsynth)
library(data.table)
library(dplyr)
library(fixest)
library(ggplot2)
library(did)


# Getting the workign directory 
shell_root <- "/scratch/gpfs/ar8787/groupdata2/india_forest_land" 
dbox_root <- "~/Dropbox/india_forest_land" 
root <- shell_root
setwd( root )

### Geting State names
state_names <- fread( "A_MicroData/state_names_shrug.csv" )
state_id_code <- {i}
st_name <- state_names[ state_names$state_id == state_id_code ]$state_name


# Getting the num of files
# Import data
df1 <- fread( "A_MicroData/data_sysdif_block_lvl.csv" )
names(df1)

df1 <- df1[df1$state_id == state_id_code ]
df_year_2010 <- df1[df1$year == 2010]


# Calculate the median of the 'value' column
median_value <- median(df_year_2010$per_treecover)
# Generate a new variable 'above_median'; it will be TRUE if 'value' is greater than the median
df_year_2010[, above_median_block_any_all := (per_treecover > median_value)*1 ]
df_year_2010 <- df_year_2010[, .(block_id, above_median_block_any_all )]

df2 <- df1 %>% left_join( df_year_2010 )

# Generation of t2event
# Create a data.table with minimum year by block_id
min_years_data <- df2[
  post_ror_data_entry_block_any == 1,
  .(min_year = min(year)), # Replace 'year_column' with your actual year column name
  by = .(block_id)
]



# Merge the min_years_data back to the original dataset
df2 <- merge(df2, min_years_data, by = "block_id", all.x = TRUE)
df2[is.na(min_year), min_year := 0]
df2$t2ev_ror_data_entry_block_any <- df2$year - df2$min_year


df2_above <- df2[df2$above_median_block_any_all == 1]
df2_below <- df2[df2$above_median_block_any_all == 0]
"""

    text2 = """
#-------------------------------------------------------------------------------
# above the median
#-------------------------------------------------------------------------------

len_above = length(unique(df2_above$min_year))

if ( len_above > 1 ){
  
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_above,
                est_method = "reg",
                control_group = "notyettreated"
  )
  
  filename <- paste0( "above_median_block_any_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_above[t2ev_ror_data_entry_block_any %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_any]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_any
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, "- Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "above_median_block_any_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)
  
}
#-------------------------------------------------------------------------------





#-------------------------------------------------------------------------------
# below the median
#-------------------------------------------------------------------------------

len_below = length(unique(df2_below$min_year))

if ( len_below > 1 ){
  
  # Start estimation
  out <- att_gt(yname = "per_treecover",
                gname = "min_year",
                idname = "block_id",
                tname = "year",
                xformla = ~ 1 ,
                data = df2_below,
                est_method = "reg",
                control_group = "notyettreated"
  )
  
  filename <- paste0( "below_median_block_any_", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  saveRDS( out, path )
  
  
  # aggregate the group-time average treatment effects
  dynamic_sum <- aggte( out, type = "dynamic")
  res <- ggdid(dynamic_sum)
  data_sum <- res$data
  data_sum$ymin=(data_sum$att-data_sum$c*data_sum$att.se)
  data_sum$ymax = ( data_sum$att + data_sum$c * data_sum$att.se )
  est_df <- data.table( data_sum )
  
  # Vectors of old and new column names
  oldnames <- c( "year", "att", "ymax", "ymin" )
  newnames <- c( "Time", "Estimate", 'upper_bound', 'lower_bound' )
  setnames(est_df, oldnames, newnames)
  
  # Getting the number of obseravtions
  n_df <- df2_below[t2ev_ror_data_entry_block_any %in% est_df$Time][, .N, by = t2ev_ror_data_entry_block_any]
  n_df$Time <- n_df$t2ev_ror_data_entry_block_any
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "blue") +
    geom_point(color = "red") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, 
                                 ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to First Year of Treatment", 
         title = "Ror Data Entry at Level") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( paste0( st_name, " - Non yet Treated and Never Treated" ) ) +
    theme_minimal()
  
  filename <- paste0( "below_median_block_any_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_callaway_santanna_analysis/median_filter/by_state", 
                    filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)
  
}
#-------------------------------------------------------------------------------
"""
    
    # Saving R scripts
    final_code = text1 + text2
    filename = f"sate_{i}_block_any"
    with open( fr"{main_path}/batch/{filename}.R", "w") as f:
        f.write( f"{final_code}")
    
    
    text = f"""#!/bin/bash
#SBATCH --nodes=1                # node count
#SBATCH --ntasks=1               # total number of tasks across all nodes
#SBATCH --cpus-per-task=1        # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --mem-per-cpu=4G         # memory per cpu-core (4G per cpu-core is default)
#SBATCH --time=08:00:00          # total run time limit (HH:MM:SS)
#SBATCH --mail-user=futurolos9@gmail.com
#SBATCH --mail-type=BEGIN
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL
#SBATCH --output={filename}.log

module purge
R CMD BATCH {main_path}/batch/{filename}.R
"""
    
    with open( f"{main_path}/batch/{filename}.sbatch", "w") as f:
            f.write( f"{text}")


In [83]:
n1 = n_sample - 1
files = np.arange(1, n_sample).reshape( 1, n1 )
names = ['anzony' ]

text1 = f"""
# submit
cd {main_path}/batch
"""
i = 0
for name in names:
    vlaset = files[ i, :]
    
    text2 = ""
    for val in states_sel:

        if val < n_sample:
            text2 = text2 + f"""
sbatch sate_{val}_block_any.sbatch
                    """
            
    with open( fr"{main_path}/anzony_block_any2.sh", "w") as f:
        f.write( f"{text1 + text2}")
    i = i + 1
